In [110]:
import json
import glob
import pandas as pd
import os

## Setup parameter


In [288]:
input_file_path ='./t2dv2-test-input'
json_save_path = './mtab_json_train'
csv_save_path = './mtab_csv_train'
gt_file_path = './round_1'
gt_label_path = './mtab_gt_train'
metrics_save_path = './mtab_metrics_train'
total_metrics = 'train_results.csv'
limit = 1000

In [275]:

!mkdir -p $json_save_path
!mkdir -p $csv_save_path
!mkdir -p $gt_label_path
!mkdir -p $metrics_save_path

## Get MTab created json file

In [260]:
ls = glob.glob(input_file_path + "/*.csv")

for file in ls:
    json_name = json_save_path + file[len(input_file_path):len(file)-4] + '.json'
    !curl -X POST -F file=@$file https://mtab.app/api/v1/mtab?limit=$limit -o $json_name

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228k  100  212k  100 16493   1644    124  0:02:13  0:02:12  0:00:01 4875016493    0     0  100 16493      0   4586  0:00:03  0:00:03 --:--:--  4586    0    312  0:00:52  0:00:52 --:--:--     09 --:--:--     01:07  0:01:06  0:00:01     0    0    149  0:01:50  0:01:49  0:00:01     0
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24538  100 22269  100  2269   1980    201  0:00:11  0:00:11 --:--:--  4665
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30358  100 27133  100  3225   2983    354  0:00:09  0:00:09 --:--:--  7489:08 --:--:--     06  3509
  % Total    % Received % Xferd  Average Speed   Time   

## Transform json to table-linker format csv

In [294]:
ls = glob.glob( json_save_path+ "/*.json")
for file in ls:
    if os.path.getsize(file) == 0:
        continue
    f = open(file)
    data = json.load(f)
    
    # get data and input from json file to arrays 
    target = data['tables'][0]['semantic']['cea']
    inp = data['tables'][0]['table_cells']


    #load the data we want
    annotation = []
    desc= []
    kg_id = []
    kg_label = []
    wikipedia = []
    target_ls = []
    tar = []
    col = []
    for i in range(0, len(target)):
        if 'dbpedia' in target[i]['annotation']:
            annotation.append(target[i]['annotation']['dbpedia'][target[i]['annotation']['dbpedia'].find('resource/')+9:])
        else:
            annotation.append('')
        desc.append(target[i]['annotation']['desc'])
        kg_label.append(target[i]['annotation']['label'])
        kg_id.append(target[i]['annotation']['wikidata'][target[i]['annotation']['wikidata'].find('/entity/')+8:])

        if 'wikipedia' in target[i]['annotation']:
            wikipedia.append(target[i]['annotation']['wikipedia'][target[i]['annotation']['wikipedia'].find('/wiki/')+6:])
        else:
            wikipedia.append('')
        tar.append(target[i]['target'])
        target_ls.append(inp[target[i]['target'][0]][target[i]['target'][1]])

    col = []
    row= []
    for i in tar:
        row.append(i[0]-1)
        col.append(i[1])

    import pandas as pd
    df = pd.DataFrame(list(zip(col, row, target_ls, kg_id, kg_label, desc, annotation,wikipedia)),
                   columns =['column', 'row', 'label', 'kg_id', 'kg_labels','kg_descriptions', 'dbpedia', 'wikipedia'])


    df.to_csv(csv_save_path + file[len(json_save_path):len(file)-5] +'.csv', index = False)
    

## Label the results with ground truth

In [295]:
ls = glob.glob(csv_save_path+ "/*.csv")
for file in ls:
    if os.path.getsize(file) == 0:
        continue
    gt_file = gt_file_path + file[len(csv_save_path):]
    output_path = gt_label_path + file[len(csv_save_path):]
    
    !tl ground-truth-labeler -f $gt_file < $file > $output_path


ground-truth-labeler Time: 0.015277862548828125s
ground-truth-labeler Time: 0.013573884963989258s
ground-truth-labeler Time: 0.011341094970703125s
ground-truth-labeler Time: 0.011157989501953125s
ground-truth-labeler Time: 0.013378143310546875s
ground-truth-labeler Time: 0.011590003967285156s
ground-truth-labeler Time: 0.019619226455688477s
ground-truth-labeler Time: 0.014224052429199219s
ground-truth-labeler Time: 0.01332712173461914s
ground-truth-labeler Time: 0.019666194915771484s
ground-truth-labeler Time: 0.012255430221557617s
ground-truth-labeler Time: 0.009737968444824219s
ground-truth-labeler Time: 0.011922121047973633s
ground-truth-labeler Time: 0.024389028549194336s
ground-truth-labeler Time: 0.010139226913452148s
ground-truth-labeler Time: 0.012969017028808594s
ground-truth-labeler Time: 0.010870933532714844s
ground-truth-labeler Time: 0.016103029251098633s
ground-truth-labeler Time: 0.008982181549072266s
ground-truth-labeler Time: 0.015986919403076172s
ground-truth-labeler 

## Get Metrics and output to file

In [296]:
ls = glob.glob( gt_label_path+ "/*.csv")
df_list = []
for file in ls:
    if os.path.getsize(file) == 0:
        continue
    output_path = metrics_save_path + file[len(gt_label_path):]
    out = file[len(gt_label_path)+1:]
    !tl metrics $file -k 1 -c "evaluation_label" --tag $out > $output_path
    if os.path.getsize(output_path) > 1:
        df = pd.read_csv(output_path)
        df_list.append(df)

pd.concat(df_list).to_csv(total_metrics, index = False)

metrics Time: 0.3472428321838379s
metrics Time: 0.3473348617553711s
metrics Time: 0.05530977249145508s
metrics Time: 0.23428082466125488s
metrics Time: 0.337691068649292s
metrics Time: 0.3560302257537842s
metrics Time: 1.1447820663452148s
metrics Time: 0.3462400436401367s
metrics Time: 0.40024805068969727s
metrics Time: 1.1870641708374023s
metrics Time: 0.5256850719451904s
metrics Time: 0.052133798599243164s
metrics Time: 0.3260002136230469s
metrics Time: 2.255105972290039s
metrics Time: 0.14565038681030273s
metrics Time: 0.6289091110229492s
metrics Time: 0.13720178604125977s
metrics Time: 0.7960202693939209s
metrics Time: 0.07692313194274902s
metrics Time: 0.3944129943847656s
metrics Time: 1.2355399131774902s
metrics Time: 0.577672004699707s
metrics Time: 0.032206058502197266s
metrics Time: 1.438291072845459s
metrics Time: 1.25022292137146s
metrics Time: 0.7288639545440674s
metrics Time: 1.0059857368469238s
metrics Time: 0.11718511581420898s
metrics Time: 0.5594990253448486s
metrics T